We're working on a test framework to write tests for the below (bad) implementation of `find_twos`. 

In [6]:
def find_twos(first_list, second_list):
    return []

So far we have written some matchers for our test framework: `has_items`, `has_size`, `is_empty`, and `equals`. For convenience, I stuck our implementations into a file called `matchers` inside a package called `phoenix_test` _right here in this folder_ (you can even go look at it!)

This next line imports all that stuff we wrote:

In [7]:
from phoenix_test.matchers import FailedAssertion, Assertion, assert_that

We used that stuff to write some tests for our mediocre-at-best implementation of `find_twos` like so:

In [8]:
    def test_empty_inputs():
        print("")
        assert_that(find_twos("", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()
        assert_that(find_twos("2", "")).is_empty()

    def test_non_matching_sets():
        assert_that(find_twos("1", "1, 3")).is_empty()

    def test_non_matching_twos():
        assert_that(find_twos("2", "1, 3")).is_empty()
        
    def test_matches():
        # This test is going to fail because our find_twos method works crappy
        # This is by design to make sure our testing framework...actually tests something
        assert_that(find_twos("12", "2, 12")).has_size(1)
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_size(2)
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).has_items(2, 22)

What happens when we run the above block of code?

Ah, yes, nothing. The reason nothing happens is that the above block of code _defines_ four test methods, but it doesn't _call_ them. In order to run these tests, we have to call our test methods like so:

In [9]:
test_empty_inputs()
test_non_matching_sets()
test_non_matching_twos()
test_matches()

FailedAssertion: Expected [] to have size 1 but it instead had size 0

But that's kind of annoying. We don't want to have to explicitly _call_ every test that we write like this. It would be annoying to have to keep a file that just lists out and calls every single test method so we can run our tests all at once...plus, that's mistake-prone, since we might write a test and forget to put it in the list, or delete one and forget to take it out.

What if our `Test` superclass were able to run all our tests for us?

In [45]:
class Test():
    # Runs all the test methods. HOW?!?!
    def run(self):
        test_methods = [
            token for token in dir(self) \
            if token.startswith("test")  \
            and callable(getattr(self.__class__, token))
        ]
        for method in test_methods:
            print(f"Running {method}.")
            try:
                getattr(self.__class__, method).__call__(self)
            except Exception as e:
                print(e) 

### Let's look at what is happening in this `run` method.

Practice making a referential map by tracing the `Test` class. Which Python functionality is it using? How do those functions work? Do you have any open questions?


### A Tool for You: Python's Built-In Documentation

Python provides you with some assistance for researching code that you are reading:

In [46]:
dir.__doc__

"dir([object]) -> list of strings\n\nIf called without an argument, return the names in the current scope.\nElse, return an alphabetized list of names comprising (some of) the attributes\nof the given object, and of attributes reachable from it.\nIf the object supplies a method named __dir__, it will be used; otherwise\nthe default dir() logic is used and returns:\n  for a module object: the module's attributes.\n  for a class object:  its attributes, and recursively the attributes\n    of its bases.\n  for any other object: its attributes, its class's attributes, and\n    recursively the attributes of its class's base classes."

In [47]:
help(dir)

Help on built-in function dir in module builtins:

dir(...)
    dir([object]) -> list of strings
    
    If called without an argument, return the names in the current scope.
    Else, return an alphabetized list of names comprising (some of) the attributes
    of the given object, and of attributes reachable from it.
    If the object supplies a method named __dir__, it will be used; otherwise
    the default dir() logic is used and returns:
      for a module object: the module's attributes.
      for a class object:  its attributes, and recursively the attributes
        of its bases.
      for any other object: its attributes, its class's attributes, and
        recursively the attributes of its class's base classes.



So the Test class is going to be our **superclass**. We can now **subclass** that Test class like so:

In [48]:
class FindTwosTest(Test):
    test_useless_attribute = None
    test_other_useless_attribute = None

    def test_empty_inputs(self):
        print("")
        assert_that(find_twos("", "")).equals([])
        assert_that(find_twos("2", "")).equals([])
        assert_that(find_twos("2", "")).equals([])

    def test_non_matching_sets(self):
        assert_that(find_twos("1", "1, 3")).equals([])

    def test_non_matching_twos(self):
        assert_that(find_twos("2", "1, 3")).equals([])
        
    def test_matches(self):
        assert_that(find_twos("12", "2, 12")).equals([12])
        assert_that(find_twos("1, 2, 20, 22, 44, 99", "3, 5, 22, 100, 44, 2")).equals([2, 22])

In [49]:
FindTwosTest().run()

Running test_empty_inputs.

Running test_matches.
Expected [12] but got []
Running test_non_matching_sets.
Running test_non_matching_twos.


### A Tool for You: Running Your Own Experiments

On _this particular_ code, I have kept it in small chunks inside of a REPL environment so that you can remove or change lines of code to investigate what they are doing.

### Challenge: 

1. What happens if you comment out `and callable(getattr(self.__class__, token))` in the list comprehension?
1. What happens if you remove the try/except above and just call `getattr(self.__class__, method).__call__(self)` right after the print statement?

### We have a test runner. Woo!

